In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, dot
from tensorflow.keras import regularizers
from sklearn.decomposition import TruncatedSVD
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
from tqdm import trange, notebook

In [2]:
user_inventory=pd.read_csv("./Data/embeding_rating.csv")
game_info=pd.read_csv("./data/game_detail_preprocessing.csv")

In [3]:
del user_inventory["Unnamed: 0"]
del game_info["Unnamed: 0"]

In [4]:
user_inventory.head(3)

Counter-Strike: Global Offensive  Grand Theft Auto V  Total War: SHOGUN 2  \
0                          0.460547            0.257275             0.107505   
1                          1.236492            0.947491             0.336702   
2                          1.227606            0.965474             0.318682   

   Path of Exile  Life is Feudal: Your Own  \
0       0.640422                  0.340797   
1       3.873230                  0.781555   
2       2.669402                  0.785304   

   Call of Duty: Modern Warfare 2 - Multiplayer  Chivalry: Medieval Warfare  \
0                                      0.449093                    0.043132   
1                                      0.982932                    0.098145   
2                                      0.982270                    0.103198   

   Call of Duty: Black Ops II  Infestation: Survivor Stories 2020  \
0                    0.037675                            0.000306   
1                    0.077848                            0.002558   
2                    0.075032                            0.002837   

   Space Engineers  ...  Sakura Space  Negligee  Escape Dead Island  \
0         0.041271  ...      0.657018  0.088584            0.150951   
1         0.108208  ...      1.339629  0.245297            0.378320   
2         0.103871  ...      1.319382  0.247969            0.376239   

   The Forgotten Ones   F1 2014  Wild Animal Racing  Luna Online: Reborn  \
0            0.535680  0.162711            0.215170             0.376517   
1            1.130005  0.372688            0.620238             1.274697   
2            1.189568  0.376436            0.591505             1.186636   

   Shantae and the Seven Sirens  Epic Battle Fantasy 4  \
0                      0.469047               0.091355   
1                      0.970412               0.213148   
2                      1.059214               0.219903   

   Danganronpa 2: Goodbye Despair  
0                        0.104336  
1                        0.291913  
2                        0.314533  

[3 rows x 602 columns]

In [5]:
game_info.head(3)

game name                                        description  \
0         Counter-Strike  Play the world's number 1 online action game. ...   
1  Team Fortress Classic  One of the most popular online action games of...   
2     Deathmatch Classic  Enjoy fast-paced multiplayer gaming with Death...   

      price                                         categories  genres  \
0  ₩ 10,500  ['Multi-player', 'PvP', 'Online PvP', 'Shared/...  Action   
1   ₩ 5,500  ['Multi-player', 'PvP', 'Online PvP', 'Shared/...  Action   
2   ₩ 5,500  ['Multi-player', 'PvP', 'Online PvP', 'Shared/...  Action   

   recommendations  
0          98065.0  
1           3664.0  
2           1250.0

In [6]:
def Collaborative_Filtering(User_Inventory,Base_Game):
    matrix=User_Inventory.values.T
    corr=np.corrcoef(matrix)
    game_name=User_Inventory.columns
    game_name_list=list(game_name)
    input_value=game_name_list.index(Base_Game)
    corr_input_value=corr[input_value]
    Recommend_result=list(game_name[corr_input_value>=0.95])[:20]
    for recommend_game in Recommend_result:
        if recommend_game== Base_Game:
            Recommend_result.remove(recommend_game)
    return Recommend_result

In [65]:
def content_based_filtering(GameData,Input_game):
    
    dic_recommend={}
    app_name=GameData["game name"]
    tfidf_vectorizer = TfidfVectorizer(analyzer="word",strip_accents = 'unicode',stop_words='english')
    tfidf = tfidf_vectorizer.fit_transform(list(GameData['description']))
    for index in range(GameData['description'].shape[0]):
        cosine_similar = linear_kernel(tfidf[index],tfidf).flatten()
        related_rating = (-cosine_similar).argsort()[:21]   
        dic_recommend.update({app_name[index]:[app_name[i] for i in related_rating]})
    content_based_results = pd.DataFrame(dic_recommend)
    content_based_results.reset_index(inplace=True)
    content_based_results = content_based_results.T
    result=content_based_results.drop(0,axis=1)
    
    return list(result.loc[Input_game])

In [192]:
def Hybrid_Filtering(User_Inventory,GameData,Base_Game):
    
    
    # Content-Based-Filtering
    cont_app_name=GameData["game name"]
    tfidf_vectorizer = TfidfVectorizer(analyzer="word",strip_accents = 'unicode',stop_words='english')
    tfidf = tfidf_vectorizer.fit_transform(list(GameData['description']))
    for index in range(GameData['description'].shape[0]):
        cosine_similar = linear_kernel(tfidf[index],tfidf).flatten()
        
    # Rating용 데이터프레임 구성 - Content_Based_Filtering
    if Base_Game in cont_app_name:
        input_value=cont_app_name.index(Base_Game)
        global cont_sim_df
        cont_sim_df=pd.DataFrame(cosine_similar[input_value])
        cont_sim_df.rename(columns={0:"rating"},inplace=True)
        cont_sim_df.insert(0,'game name',cont_app_name)
        
        
    # Collaborative-Filtering
    
    matrix=User_Inventory.values.T
    corr=np.corrcoef(matrix)
    game_name=User_Inventory.columns
    game_name_list=list(game_name)

    # Rating용 데이터프레임 구성 - Collaborative-Filtering
    
    if Base_Game in game_name_list:
        input_value=game_name_list.index(Base_Game)
        corr_input_value=corr[input_value]
        corr_input_value=corr_input_value
        global col_corr_d
        col_corr_df=pd.DataFrame(corr_input_value)
        col_corr_df.rename(columns={0:"rating"},inplace=True)
        col_corr_df.insert(0,'game name',game_name_list)
    
    # Weighted Rating for Hybried
    rating_lst=[]
    gameName_set=set()
    
    for i in range(len(cont_sim_df["game name"])):
        gameName_set.add(cont_sim_df["game name"][i])
        found=0
        for j in range(len(col_corr_df["game name"])):
            if cont_sim_df["game name"][i]==col_corr_df["game name"][j]:
                temp_rating=0.9*cont_sim_df["rating"][i]+col_corr_df["rating"][j]
                rating_lst.append(temp_rating)
                break
            
        if found==0:
            temp_rating=0.9*cont_sim_df["rating"][i]
            rating_lst.append(temp_rating)
            
    for i in range(len(col_corr_df["game name"])):
        if col_corr_df["game name"][i] not in gameName_set:
            gameName_set.add(col_corr_df["game name"][i])
            temp_rating=0.95*col_corr_df["rating"][i]
            rating_lst.append(temp_rating)
        
        
    gameName_lst=[]
    result_recom=[]
    
    for i in gameName_set:
        gameName_lst.append(i)
    
    rating_lst=np.asarray(rating_lst)
    
    recom_index=(-1*rating_lst).argsort()[:10]
    for i in recom_index:
        result_recom.append(gameName_lst[i])

    print("<Recommended Game List>\n")
    for i in range(len(result_recom)):
        print(i+1,"위\t",result_recom[i],end="\n")
    

In [193]:
Hybrid_Filtering(user_inventory,game_info,"Counter-Strike")

<Recommended Game List>

1 위	 Destiny Warriors RPG
2 위	 Rake
3 위	 Insincere
4 위	 We Were Here
5 위	 Runestone Keeper
6 위	 The Incredible Adventures of Van Helsing: Final Cut
7 위	 Warhammer 40,000: Dawn of War II - Chaos Rising
8 위	 Frostpunk
9 위	 Bloons TD 5
10 위	 Bejeweled 2 Deluxe


In [10]:
def Hybrid_Filtering_2(User_Inventory,GameData,Base_Game):
    Col_GameList=Collaborative_Filtering(User_Inventory,Base_Game)
    Con_GameList=content_based_filtering(GameData,Base_Game)
    result=[]
    Col_recom_index=[]
    Con_recom_index=[]
    cnt=0
    finish=False
    
    while(finish):
        for i in notebook.tqdm(range(len(Col_GameList))):
            for j in range(len(Con_GameList)):
                if Col_GameList[i]==Con_GameList[j]:
                    result.append(Con_GameList[j])
                    Col_recom_index.append(i)
                    Con_recom_index.append(j)
                    cnt+=1
        finsih=True
    
    if cnt<10:
        while(cnt<10):
            for i in range(10):
                if i not in Col_recom_index:
                    result.append(Col_GameList[i])
                    cnt+=1
                if cnt<10:
                    result.append(Con_GameList[i])
                    cnt+=1
                else:
                    break
        
    print("<Recommended Game List>\n")
    result=result[0:10]
    for i in range(len(result)):
        print(i+1,"위\t",result[i],end="\n")

In [11]:
Hybrid_Filtering_2(user_inventory,game_info,"Counter-Strike")

<Recommended Game List>

1 위	 Counter-Strike: Global Offensive
2 위	 Team Fortress Classic
3 위	 Grand Theft Auto V
4 위	 Insurgency
5 위	 Total War: SHOGUN 2
6 위	 Tom Clancy's Rainbow Six® 3 Gold
7 위	 Path of Exile
8 위	 Call of Duty® 4: Modern Warfare®
9 위	 Life is Feudal: Your Own
10 위	 F.E.A.R.
